In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the VGG model architecture for CIFAR-10 with added Dropout
class VGG(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),  # Add Dropout here and in other layers
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Define data transforms
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load CIFAR-10 dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# Initialize the VGG model, loss function, and Adam optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = VGG(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)  # Use Adam optimizer

# Training loop
for epoch in range(200):  # Adjust the number of epochs as needed
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training')

# Evaluation
net.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')


Files already downloaded and verified
Files already downloaded and verified
[1,   200] loss: 2.192
[1,   400] loss: 2.028
[1,   600] loss: 1.918
[2,   200] loss: 1.855
[2,   400] loss: 1.827
[2,   600] loss: 1.787
[3,   200] loss: 1.686
[3,   400] loss: 1.637
[3,   600] loss: 1.600
[4,   200] loss: 1.516
[4,   400] loss: 1.481
[4,   600] loss: 1.446
[5,   200] loss: 1.371
[5,   400] loss: 1.360
[5,   600] loss: 1.303
[6,   200] loss: 1.260
[6,   400] loss: 1.242
[6,   600] loss: 1.219
[7,   200] loss: 1.158
[7,   400] loss: 1.140
[7,   600] loss: 1.127
[8,   200] loss: 1.087
[8,   400] loss: 1.062
[8,   600] loss: 1.033
[9,   200] loss: 0.998
[9,   400] loss: 1.007
[9,   600] loss: 0.972
[10,   200] loss: 0.940
[10,   400] loss: 0.928
[10,   600] loss: 0.925
[11,   200] loss: 0.908
[11,   400] loss: 0.888
[11,   600] loss: 0.871
[12,   200] loss: 0.842
[12,   400] loss: 0.829
[12,   600] loss: 0.837
[13,   200] loss: 0.809
[13,   400] loss: 0.788
[13,   600] loss: 0.789
[14,   200] los

[113,   600] loss: 0.199
[114,   200] loss: 0.188
[114,   400] loss: 0.209
[114,   600] loss: 0.186
[115,   200] loss: 0.180
[115,   400] loss: 0.187
[115,   600] loss: 0.184
[116,   200] loss: 0.190
[116,   400] loss: 0.215
[116,   600] loss: 0.204
[117,   200] loss: 0.226
[117,   400] loss: 0.219


KeyboardInterrupt: 